In [1]:
from sklearn.metrics import accuracy_score, roc_auc_score
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from keras.optimizers import SGD
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

2023-06-01 20:39:03.756373: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import tensorflow as tf
import numpy as np
from typing import Any
def one_hot_encode(sequence: str,
                   alphabet: str = 'ACGT',
                   neutral_alphabet: str = 'N',
                   neutral_value: Any = 0,
                   dtype=np.float64) -> np.ndarray:
  """One-hot encode sequence."""
  def to_uint8(string):
    return np.frombuffer(string.encode('ascii'), dtype=np.uint8)
  hash_table = np.zeros((np.iinfo(np.uint8).max, len(alphabet)), dtype=dtype)
  hash_table[to_uint8(alphabet)] = np.eye(len(alphabet), dtype=dtype)
  hash_table[to_uint8(neutral_alphabet)] = neutral_value
  hash_table = hash_table.astype(dtype)
  return hash_table[to_uint8(sequence)]

In [3]:
filename = '/home/xlxiang/biostatistics/data/Classification/pos.fa'
f = open(filename)
lines = f.readlines()
f.close()
pos_seq = [lines[i].strip() for i in range(len(lines)) if i  % 2 != 0]

In [4]:
filename = '/home/xlxiang/biostatistics/data/Classification/neg.fa'
f = open(filename)
lines = f.readlines()
f.close()
neg_seq = [lines[i].strip() for i in range(len(lines)) if i  % 2 != 0]

In [5]:
pos_matrix = [one_hot_encode(i) for i in pos_seq]
neg_matrix = [one_hot_encode(i) for i in neg_seq]
pos_label = np.ones([6606, 1])
neg_label = np.zeros([6606, 1])
input_x = np.row_stack((pos_matrix, neg_matrix))
input_y = np.row_stack((pos_label, neg_label))
print(input_x.shape, input_y.shape)

(13212, 500, 4) (13212, 1)


In [6]:
def Model(a,b):
    model = Sequential()
    model.add(
        Conv1D(filters=int(b),
               kernel_size=int(a),
               padding='same',
               activation='relu',
               input_shape=(500, 4)))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input_x,
                                                    input_y,
                                                    train_size=0.8,
                                                    random_state=2**10 * 6)
x_train, x_valid, y_train, y_valid = train_test_split(x_train,
                                                    y_train,
                                                    train_size=0.9,
                                                    random_state=2**10 * 6)
print(len(x_train))
print(len(x_valid))
print(len(x_test))

9512
1057
2643


In [8]:
batch_size = 64
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
dataset_valid = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

2023-06-01 20:41:40.844443: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-01 20:41:40.845624: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-01 20:41:41.113268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-01 20:41:41.113923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6575GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2023-06-01 20:41:41.113950: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-01 20:41:41.115996: I tensorflow/stream_executor/platform/defa

In [9]:
def matrix(y_test_all, y_pred_all):
    y_test_all = np.array(y_test_all)
    y_pred_all = np.array(y_pred_all)

    auc = roc_auc_score(y_test_all, y_pred_all)

    y_pred_all[y_pred_all >= 0.5] = 1
    y_pred_all[y_pred_all < 0.5] = 0
    y_test_all.ravel()
    y_test_all.ravel()
    acc = accuracy_score(y_test_all, y_pred_all)
    return acc, auc

In [10]:
best_auc = 0
for filter_len in range(7,21,2):
    for filter_num in [8,16,32,64]:
        model = Model(filter_len,filter_num)
        sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
        model.compile(loss='binary_crossentropy',
                      optimizer=sgd,
                      metrics=['accuracy'])
        checkpoint = ModelCheckpoint(filepath= '/home/xlxiang/biostatistics/code/classification/deepbind.h5',
                                     save_best_only=False,
                                     monitor='val_accuracy',
                                     mode='max')
        early_stopping =EarlyStopping(monitor='val_accuracy', patience=5)
        model.fit(dataset_train,
                  epochs=100,
                  validation_data=dataset_valid,
                callbacks=[early_stopping,checkpoint])
        print('model fit over!')
        yt_pred = []
        y_test_all = []
        for i in dataset_test:
            y_test_all.extend(i[1].numpy().ravel().tolist())
            yt_pred.extend(model.predict(i[0]).ravel().tolist())
        acc,auc = matrix(y_test_all, yt_pred)
        if auc>best_auc:
              best_auc=auc
              best_filter_len = filter_len
              best_filter_num = filter_num
print("the best filter_len: {}, the best filter_num: {}.".format(best_filter_len,best_filter_num))

Epoch 1/100


2023-06-01 20:41:42.343399: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-06-01 20:41:42.346281: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3696000000 Hz
2023-06-01 20:41:42.427009: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-06-01 20:41:42.618464: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-06-01 20:41:43.374323: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2023-06-01 20:41:43.428940: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


149/149 [==============================] - 4s 15ms/step - loss: 0.7176 - accuracy: 0.5089 - val_loss: 0.6952 - val_accuracy: 0.4825
Epoch 2/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6992 - accuracy: 0.5052 - val_loss: 0.6919 - val_accuracy: 0.5128
Epoch 3/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6930 - accuracy: 0.5283 - val_loss: 0.6920 - val_accuracy: 0.5005
Epoch 4/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6905 - accuracy: 0.5298 - val_loss: 0.6904 - val_accuracy: 0.5279
Epoch 5/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6886 - accuracy: 0.5397 - val_loss: 0.6892 - val_accuracy: 0.5270
Epoch 6/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6868 - accuracy: 0.5428 - val_loss: 0.6895 - val_accuracy: 0.5307
Epoch 7/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6833 - accuracy: 0.5556 - val_loss: 0.6868 - val_accuracy: 0.5449
Epoch 8/10

Epoch 9/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6848 - accuracy: 0.5493 - val_loss: 0.6832 - val_accuracy: 0.5743
Epoch 10/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6810 - accuracy: 0.5706 - val_loss: 0.6807 - val_accuracy: 0.5733
Epoch 11/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6799 - accuracy: 0.5750 - val_loss: 0.6777 - val_accuracy: 0.5885
Epoch 12/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6743 - accuracy: 0.5798 - val_loss: 0.6759 - val_accuracy: 0.5989
Epoch 13/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6724 - accuracy: 0.5851 - val_loss: 0.6723 - val_accuracy: 0.6036
Epoch 14/100
149/149 [==============================] - 1s 3ms/step - loss: 0.6687 - accuracy: 0.5924 - val_loss: 0.6706 - val_accuracy: 0.5970
Epoch 15/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6663 - accuracy: 0.6015 - val_loss: 0.6679 - val_accuracy: 

Epoch 15/100
149/149 [==============================] - 1s 4ms/step - loss: 0.5946 - accuracy: 0.6910 - val_loss: 0.5824 - val_accuracy: 0.6991
Epoch 16/100
149/149 [==============================] - 1s 4ms/step - loss: 0.5764 - accuracy: 0.7057 - val_loss: 0.5627 - val_accuracy: 0.7247
Epoch 17/100
149/149 [==============================] - 1s 4ms/step - loss: 0.5503 - accuracy: 0.7312 - val_loss: 0.5366 - val_accuracy: 0.7408
Epoch 18/100
149/149 [==============================] - 1s 4ms/step - loss: 0.5177 - accuracy: 0.7587 - val_loss: 0.5104 - val_accuracy: 0.7531
Epoch 19/100
149/149 [==============================] - 1s 3ms/step - loss: 0.4951 - accuracy: 0.7680 - val_loss: 0.5027 - val_accuracy: 0.7569
Epoch 20/100
149/149 [==============================] - 1s 4ms/step - loss: 0.4697 - accuracy: 0.7876 - val_loss: 0.4732 - val_accuracy: 0.7682
Epoch 21/100
149/149 [==============================] - 1s 4ms/step - loss: 0.4512 - accuracy: 0.7924 - val_loss: 0.4590 - val_accuracy:

...

Epoch 10/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6655 - accuracy: 0.5998 - val_loss: 0.6715 - val_accuracy: 0.5922
Epoch 11/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6586 - accuracy: 0.6134 - val_loss: 0.6720 - val_accuracy: 0.5847
Epoch 12/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6532 - accuracy: 0.6156 - val_loss: 0.6684 - val_accuracy: 0.5875
Epoch 13/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6509 - accuracy: 0.6292 - val_loss: 0.6650 - val_accuracy: 0.5922
Epoch 14/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6411 - accuracy: 0.6406 - val_loss: 0.6561 - val_accuracy: 0.6121
Epoch 15/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6303 - accuracy: 0.6552 - val_loss: 0.6503 - val_accuracy: 0.6178
Epoch 16/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6170 - accuracy: 0.6575 - val_loss: 0.6267 - val_accuracy:

Epoch 19/100
149/149 [==============================] - 1s 4ms/step - loss: 0.5567 - accuracy: 0.7231 - val_loss: 0.5530 - val_accuracy: 0.7209
Epoch 20/100
149/149 [==============================] - 1s 4ms/step - loss: 0.5262 - accuracy: 0.7487 - val_loss: 0.5349 - val_accuracy: 0.7181
Epoch 21/100
149/149 [==============================] - 1s 4ms/step - loss: 0.4972 - accuracy: 0.7669 - val_loss: 0.5100 - val_accuracy: 0.7427
Epoch 22/100
149/149 [==============================] - 1s 4ms/step - loss: 0.4662 - accuracy: 0.7822 - val_loss: 0.4956 - val_accuracy: 0.7512
Epoch 23/100
149/149 [==============================] - 1s 4ms/step - loss: 0.4421 - accuracy: 0.7991 - val_loss: 0.4807 - val_accuracy: 0.7635
Epoch 24/100
149/149 [==============================] - 1s 4ms/step - loss: 0.4142 - accuracy: 0.8196 - val_loss: 0.4749 - val_accuracy: 0.7682
Epoch 25/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3948 - accuracy: 0.8281 - val_loss: 0.4309 - val_accuracy:

Epoch 34/100
149/149 [==============================] - 1s 4ms/step - loss: 0.1230 - accuracy: 0.9610 - val_loss: 0.4333 - val_accuracy: 0.8382
Epoch 35/100
149/149 [==============================] - 1s 4ms/step - loss: 0.1292 - accuracy: 0.9561 - val_loss: 0.4155 - val_accuracy: 0.8496
Epoch 36/100
149/149 [==============================] - 1s 4ms/step - loss: 0.1050 - accuracy: 0.9688 - val_loss: 0.4249 - val_accuracy: 0.8524
model fit over!
Epoch 1/100
149/149 [==============================] - 3s 13ms/step - loss: 0.7262 - accuracy: 0.5092 - val_loss: 0.6968 - val_accuracy: 0.4995
Epoch 2/100
149/149 [==============================] - 1s 3ms/step - loss: 0.6957 - accuracy: 0.5191 - val_loss: 0.6949 - val_accuracy: 0.4863
Epoch 3/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6950 - accuracy: 0.5139 - val_loss: 0.6930 - val_accuracy: 0.5213
Epoch 4/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6942 - accuracy: 0.5202 - val_loss: 0.6929 - 

Epoch 8/100
149/149 [==============================] - 1s 3ms/step - loss: 0.6745 - accuracy: 0.5812 - val_loss: 0.6774 - val_accuracy: 0.5733
Epoch 9/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6677 - accuracy: 0.6030 - val_loss: 0.6762 - val_accuracy: 0.5875
Epoch 10/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6683 - accuracy: 0.6001 - val_loss: 0.6728 - val_accuracy: 0.5979
Epoch 11/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6634 - accuracy: 0.6096 - val_loss: 0.6717 - val_accuracy: 0.6045
Epoch 12/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6617 - accuracy: 0.6088 - val_loss: 0.6705 - val_accuracy: 0.6093
Epoch 13/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6543 - accuracy: 0.6252 - val_loss: 0.6680 - val_accuracy: 0.6055
Epoch 14/100
149/149 [==============================] - 0s 3ms/step - loss: 0.6509 - accuracy: 0.6303 - val_loss: 0.6658 - val_accuracy: 0

Epoch 22/100
149/149 [==============================] - 1s 4ms/step - loss: 0.4000 - accuracy: 0.8261 - val_loss: 0.4319 - val_accuracy: 0.7956
Epoch 23/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3763 - accuracy: 0.8385 - val_loss: 0.4065 - val_accuracy: 0.8174
Epoch 24/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3540 - accuracy: 0.8590 - val_loss: 0.3999 - val_accuracy: 0.8259
Epoch 25/100
149/149 [==============================] - 1s 3ms/step - loss: 0.3358 - accuracy: 0.8605 - val_loss: 0.3865 - val_accuracy: 0.8354
Epoch 26/100
149/149 [==============================] - 1s 3ms/step - loss: 0.3172 - accuracy: 0.8750 - val_loss: 0.3752 - val_accuracy: 0.8373
Epoch 27/100
149/149 [==============================] - 0s 3ms/step - loss: 0.2999 - accuracy: 0.8767 - val_loss: 0.3721 - val_accuracy: 0.8401
Epoch 28/100
149/149 [==============================] - 1s 4ms/step - loss: 0.2881 - accuracy: 0.8869 - val_loss: 0.3866 - val_accuracy:

## the best filter_len: 19, the best filter_num: 64.

In [11]:
model = Model(11,64)
sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath= '/home/xlxiang/biostatistics/code/classification/deepbind.h5',
                             save_best_only=True,
                             monitor='val_accuracy',
                             mode='max')
early_stopping =EarlyStopping(monitor='val_accuracy', patience=5)
model.fit(dataset_train,
          epochs=100,
          validation_data=dataset_valid,
        callbacks=[early_stopping,checkpoint])
print('model fit over!')

Epoch 1/100
149/149 [==============================] - 1s 5ms/step - loss: 0.6966 - accuracy: 0.5041 - val_loss: 0.6897 - val_accuracy: 0.5430
Epoch 2/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6889 - accuracy: 0.5400 - val_loss: 0.6831 - val_accuracy: 0.5705
Epoch 3/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6836 - accuracy: 0.5559 - val_loss: 0.6819 - val_accuracy: 0.5478
Epoch 4/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6755 - accuracy: 0.5817 - val_loss: 0.6758 - val_accuracy: 0.5743
Epoch 5/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6708 - accuracy: 0.5904 - val_loss: 0.6746 - val_accuracy: 0.5724
Epoch 6/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6632 - accuracy: 0.6114 - val_loss: 0.6685 - val_accuracy: 0.5989
Epoch 7/100
149/149 [==============================] - 1s 4ms/step - loss: 0.6574 - accuracy: 0.6157 - val_loss: 0.6691 - val_accuracy: 0.5951

In [12]:
#计算训练集和测试集的acc、precision、recall、f1 score、auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
def calculate_result(model,x_train, y_train,x_test, y_test):
    train_pro = model.predict(x_train)
    test_pro  = model.predict(x_test)
    train_predict = np.array(train_pro)
    test_predict = np.array(test_pro)

    train_predict[train_predict >= 0.5] = 1
    train_predict[train_predict < 0.5] = 0
    test_predict[test_predict >= 0.5] = 1
    test_predict[test_predict < 0.5] = 0
    
    train_acc = accuracy_score(y_train, train_predict)
    train_precision = precision_score(y_train, train_predict)
    train_recall = recall_score(y_train, train_predict)
    train_f1 = f1_score(y_train, train_predict)
    train_auc = roc_auc_score(y_train, train_pro)

    test_acc = accuracy_score(y_test, test_predict)
    test_precision = precision_score(y_test, test_predict)
    test_recall = recall_score(y_test, test_predict)
    test_f1 = f1_score(y_test, test_predict)
    test_auc = roc_auc_score(y_test, test_pro)

    print("train_acc: {0:.3f}, test_acc: {1:.3f}".format(train_acc, test_acc))
    print("train_pre: {0:.3f}, test_pre: {1:.3f}".format(train_precision, test_precision))
    print("train_recall: {0:.3f}, test_recall: {1:.3f}".format(train_recall, test_recall))
    print("train_f1: {0:.3f}, test_f1: {1:.3f}".format(train_f1, test_f1))
    print("train_auc: {0:.3f}, test_auc: {1:.3f}".format(train_auc, test_auc))
    return None

In [13]:
calculate_result(model,x_valid, y_valid,x_test, y_test)

train_acc: 0.843, test_acc: 0.854
train_pre: 0.856, test_pre: 0.857
train_recall: 0.837, test_recall: 0.848
train_f1: 0.847, test_f1: 0.852
train_auc: 0.920, test_auc: 0.930
